In [139]:
#Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import tarfile

#For ROC_curve and AUC implementation
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [140]:
#Loading the Data from Github Repository
username = "harisx91"
path = "Recommender_System_DAV_6300/master/Module_1/M1_Rating.csv"
df = pd.read_csv(f'https://raw.githubusercontent.com/{username}/{path}')

In [141]:

df.head()

,Users,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5
0,1,1.0,NaN,NaN,3.0,2
1,2,NaN,1.0,NaN,4.0,5
2,3,3.0,3.0,2.0,NaN,5
3,4,NaN,NaN,3.0,5.0,3
4,5,2.0,4.0,NaN,3.0,5


In [142]:
df.shape

(5, 6)

EDA

In [143]:
df.describe()

,Users,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5
count,5.000000,3.0,3.000000,2.000000,4.000000,5.000000
mean,3.000000,2.0,2.666667,2.500000,3.750000,4.000000
std,1.581139,1.0,1.527525,0.707107,0.957427,1.414214
min,1.000000,1.0,1.000000,2.000000,3.000000,2.000000
25%,2.000000,1.5,2.000000,2.250000,3.000000,3.000000
50%,3.000000,2.0,3.000000,2.500000,3.500000,5.000000
75%,4.000000,2.5,3.500000,2.750000,4.250000,5.000000
max,5.000000,3.0,4.000000,3.000000,5.000000,5.000000


In [144]:
df03a = df.copy()

def top(x):
    x.set_index('Users', inplace=True)
    df03a = pd.DataFrame({'1st Max':[],'Max1Value':[],'2nd Max':[],'Max2Value':[],'3rd Max':[],'Max3Value':[]})
    df03a.index.name='User'
    df03a.loc[x.index.values[0],['1st Max', '2nd Max','3rd Max']] = x.sum().nlargest(3).index.tolist()
    df03a.loc[x.index.values[0],['Max1Value', 'Max2Value','Max3Value']] = x.sum().nlargest(3).values
    return df03a

df_top = df03a.groupby('Users').apply(top).reset_index(level=1, drop=True).reset_index()
df_top

KeyError: "None of ['Users'] are in the columns"

In [145]:
df_01 = df.fillna(0)
df_01

,Users,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5
0,1,1.0,0.0,0.0,3.0,2
1,2,0.0,1.0,0.0,4.0,5
2,3,3.0,3.0,2.0,0.0,5
3,4,0.0,0.0,3.0,5.0,3
4,5,2.0,4.0,0.0,3.0,5


In [146]:
df_02 = df_01.copy()
df_02.index = np.arange(1, len(df_01) +1)
df_02

,Users,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5
1,1,1.0,0.0,0.0,3.0,2
2,2,0.0,1.0,0.0,4.0,5
3,3,3.0,3.0,2.0,0.0,5
4,4,0.0,0.0,3.0,5.0,3
5,5,2.0,4.0,0.0,3.0,5


In [147]:
#Loading the Data from Github Repository
username = "harisx91"
path = "Recommender_System_DAV_6300/master/Module_1/MovieID.csv"
movie = pd.read_csv(f'https://raw.githubusercontent.com/{username}/{path}')
movie

,movieID,movie
0,1001,Movie 1
1,1002,Movie 2
2,1003,Movie 3
3,1004,Movie 4
4,1005,Movie 5


In [148]:
df_3 = df_02.drop(['Users'], axis=1)

df_survey = df_3.stack().reset_index()
df_survey.columns=['Users', 'movie', 'rating']

In [149]:
df_survey.head()

,Users,movie,rating
0,1,Movie 1,1.0
1,1,Movie 2,0.0
2,1,Movie 3,0.0
3,1,Movie 4,3.0
4,1,Movie 5,2.0


In [150]:
# merge together
ratings_1 = pd.merge(left=df_survey, right=movie, how='outer')

ratings_1.sort_values(by='Users')

ratings = ratings_1[['Users','movieID','rating']]

In [151]:
# change data type
ratings['rating'] = ratings['rating'].astype('int64') 
ratings.dtypes

<ipython-input-151-72ed4a6a4e5e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['rating'] = ratings['rating'].astype('int64')


Users      int64
movieID    int64
rating     int64
dtype: object

In [152]:
n_ratings = len(ratings)
n_movies = ratings['movieID'].nunique()
n_users = ratings['Users'].nunique()

print(f"Number of ratings: {n_ratings}")
print(f"Number of unique movieId's: {n_movies}")
print(f"Number of unique users: {n_users}")
print(f"Average number of ratings per user: {round(n_ratings/n_users, 2)}")
print(f"Average number of ratings per movie: {round(n_ratings/n_movies, 2)}")

Number of ratings: 25
Number of unique movieId's: 5
Number of unique users: 5
Average number of ratings per user: 5.0
Average number of ratings per movie: 5.0


In [153]:
"""
print_user_summary
Prints the users top 3 movies
Parameters:
    the user
"""

def print_user_summary(user):
    # the header
    print("*****USER SUMMARY FOR USER %d*****" %(user))
    user_x = df.loc[user-1]
    print(user_x)

In [154]:
print_user_summary(1)

*****USER SUMMARY FOR USER 1*****
Users      1.0
Movie 1    1.0
Movie 2    NaN
Movie 3    NaN
Movie 4    3.0
Movie 5    2.0
Name: 0, dtype: float64


In [155]:
movie_features = ratings.pivot(index='Users',columns='movieID',values='rating').fillna(0)
mu_matrix = np.array(movie_features.values, dtype=int)
mu_matrix

array([[1, 0, 0, 3, 2],
       [0, 1, 0, 4, 5],
       [3, 3, 2, 0, 5],
       [0, 0, 3, 5, 3],
       [2, 4, 0, 3, 5]])

In [156]:
"""
simularity matrix
    returns a matrix of users that are similar to our user and how similar they are.
    [ (user similarity, user), (user similarity, user), .... , ]
    usings [fn] to calculate simularity
    and limits the matrix to size clusters
"""
def simularity_matrix(user, fn, clusters):
    # get the distances 
    # sort and reverse the data (closest -> least closest) and attach a bound 
    toReturn = [(fn(user, other),other) for other in range(0,len(mu_matrix)-1) if other != user]
    toReturn.sort(reverse=True)
    toReturn = toReturn[0:clusters] 
    return toReturn

In [157]:
import operator,collections

"""
filter_results
sorts all the recommended movies and returns an array of the top (num_of_recommendations)
parameters:
    all_outputs: a dictionary consisting of ALL recommendations
    num_of_recommendations: the number of recommendations we want
"""
def filter_results(all_outputs, num_of_recommendations):
    # sort the recommedations by possible rankings
    sorted_recommendations = sorted(all_outputs.items(), key=operator.itemgetter(1),reverse=True)
    # return the top n
    sorted_recommendations = sorted_recommendations[0:num_of_recommendations]
    return sorted_recommendations

In [158]:
"""
recommend
Recommends [k] number of movies using [fn] with [depth] as the depth and on user [user].
Parameters:
    user: the user we are recommending to
    depth: how many users are we comparing to this user
    fn: the function used for calculating simularity and distance
    k: the number of recommendations 
"""
def recommend (user, clusters, fn, k):
    print("############ RECOMMENDATION FOR USER %d ############" %(user))
    print("Using %s for calculating similarity/distance with %d clusters" %(fn,clusters))
    # we want to use our simularity functions to develop a list of ratings that our user might rate a movie as.
    # this allows us to think what our user will like the most in comparison
    distances = simularity_matrix(user,fn,clusters)
    #for each distance 
    all_unseen_movies = {}
    for dist,o in distances: 
        # each movie is an index, the number of movies is the LENGTH of the items row
        for i in range(0, len(mu_matrix[o])-1):
            predicted_rank = int(dist*mu_matrix[o][i])
            # update recommendations i to be the value of the similarity predicted rank 
            if (mu_matrix[o][i] > 0 and mu_matrix[user][i] <= 0): #if they ranked it but we didn't
                # if we are already planning to recommend it, add more weight to the prediction and increase similarity!
                all_unseen_movies[i] = (dist, [predicted_rank]) if i not in all_unseen_movies else (all_unseen_movies[i][0] + dist, all_unseen_movies[i][1] + [predicted_rank])   
    # update all the values
    for m in all_unseen_movies: 
        all_unseen_movies[m] = sum(all_unseen_movies[m][1])/all_unseen_movies[m][0] if all_unseen_movies[m][0] != 0 else 0 
    # at this point we have a dictionary of all possible recommendations with the format:
    # { [movie_id] : [predicted_rating]}
    # lets pick the top 5!
    recommend_movies = filter_results(all_unseen_movies,k)
    # recommend movies outputs an array formatted like so: [ (movie_id, predicted rank)]
    # lets print it in a nice way:
    print_recommendations(recommend_movies)
    print("############ END OF RECOMMENDATION FOR USER %d ############" %(user))
    return recommend_movies

In [159]:
"""
Jaccard Similarity
computes the jaccard simularity function on the users and returns the value
PArameters: the users we are comparing
"""
def jaccard_similarity(user1, user2):
    # jacard similarity is the intersection of the 2 / the union of the 2 
    # in other words, the common rankings / all ranked
    # we just care about length, so don't worry about maintaing indices
    union_ab = []
    intersection_ab = []
    for i in range (0, len(mu_matrix[user1]-1)):
        if mu_matrix[user1][i] > 0 and mu_matrix[user2][i] > 0:
            intersection_ab.append(mu_matrix[user1][i])
        else: # can't be both
            if mu_matrix[user1][i] > 0:
                union_ab.append(mu_matrix[user1][i])
            elif mu_matrix[user2][i] > 0:
                union_ab.append(mu_matrix[user2][i])    
    # typically jaccard only factors in set difference
    # for the sake of the movie example, we will use the actual ratings to see if this makes it more accurate
    return 1 - (sum(intersection_ab)/sum(union_ab)) if sum(union_ab) != 0 else 0

In [160]:
# top 2 recommendations for user1 by using jaccard_similarity
recommend(1, 2, jaccard_similarity, 2)

############ RECOMMENDATION FOR USER 1 ############
Using <function jaccard_similarity at 0x7fb3ba397790> for calculating similarity/distance with 2 clusters


NameError: name 'print_recommendations' is not defined